# DGEMM for KNL

Reproduction of:
Lim, R., Lee, Y., Kim, R. et al. Cluster Comput (2018) 21: 1785. https://doi.org/10.1007/s10586-018-2810-y

In [ ]:
require 'BOAST'
include BOAST
set_lang(C)
set_array_start(0)
type_map = { 4 => NArray::SFLOAT, 8 => NArray::FLOAT}

## Micro-kernel
### Definition
The first step is implementing an efficient micro-kernel
The micro-kernel update a block of $C$ of size $[mr, nr]$ noted $\widehat{C}$ using a block of $A$ of size $[mr, kb]$ noted $\widehat{A}$ and a block of $B$ of size $[kb, nr]$ noted $\widehat{B}$. $\widehat{A}$ is stored in column major order while $\widehat{C}$ and $\widehat{B}$ are stored in row major order.

In [ ]:
def micro_kernel(vector_length: 4, nr: nil, mr: nil, kb: nil)
  raise "nr must be a multiple of vector_length!" unless nr % vector_length == 0
  nvec = nr / vector_length
  register_number = mr * nvec + nvec
  puts "Using #{register_number} registers..."
  
  ah = Real :ah, dim: [Dim(mr), Dim(kb)], dir: :in, restrict: true
  bh = Real :bh, vector_length: vector_length, dim: [Dim(nvec), Dim(kb)], dir: :in, restrict: true
  ch = Real :ch, vector_length: vector_length, dim: [Dim(nvec), Dim(mr)], dir: :inout, restrict: true
  inp = Procedure("inp_#{vector_length}_#{nr}_#{mr}_#{kb}", [ah, bh, ch]) {
    regs = (0...nvec).collect { |n|
      (0...mr).collect { |m|
        Real :"reg_#{n}_#{m}", vector_length: vector_length, register: true
      }
    }
    regs_b = (0...nvec).collect { |n|
      Real :"regs_b_#{n}", vector_length: vector_length, register: true
    }
    decl *regs.flatten
    decl *regs_b
    (0...mr).collect { |m|
      (0...nvec).collect { |n|
        pr regs[n][m] === ch[n, m]
      }
    }
    i = Int :i
    decl i
    pr For(i, 0, kb - 1) {
      (0...nvec).each { |n|
        pr regs_b[n] === bh[n, i]
      }
      (0...mr).each { |m|
        (0...nvec).each { |n|
          pr regs[n][m] === FMA(Set(ah[m, i], regs_b[0]), regs_b[n], regs[n][m])
        }
      }
    }
    (0...mr).collect { |m|
      (0...nvec).collect { |n|
        pr ch[n, m] === regs[n][m]
      }
    }
  }
  inp.ckernel(includes: "immintrin.h")
end

### Test

In [ ]:
vector_length = 4
mr = 7
nr = 8
kb = 480
nvec = nr / vector_length
type = type_map[get_default_real_size]
alignment = get_default_real_size*vector_length
a = NMatrix::new(type, kb, mr).random!
b = NMatrix::new(type, nr, kb).random!
c = NMatrix::new(type, nr, mr).random!

ah = ANArray::new(type, alignment, mr, kb)
bh = ANArray::new(type, alignment, vector_length, nvec, kb)
ch = ANArray::new(type, alignment, vector_length, nvec, mr)
c_ref = ANArray::new(type, alignment, vector_length, nvec, mr)
ah[true, true] = a.transpose(1,0)[true, true]
bh[true, true, true] = b.reshape(vector_length, nvec, kb)[true, true, true]
ch[true, true, true] = c.reshape(vector_length, nvec, mr)[true, true, true]
c_ref[true, true, true] = (a*b + c).reshape(vector_length, nvec, mr)[true, true, true]
nil

In [ ]:
p = micro_kernel(vector_length: vector_length, mr: mr, nr: nr, kb: kb)
p.run(ah, bh, ch)

In [ ]:
max_error = (ch - c_ref).abs.max
raise "Computation error!" if max_error > 1e-8

In [ ]:
p.run(ah, bh, ch)
repeat_inner = 100
res = 1000.times.collect {
  p.run(ah, bh, ch, repeat: repeat_inner)
}
best = res.min { |r1, r2|
  r1[:duration] <=> r2[:duration]
}
perf = mr * nr * kb * 2 / (best[:duration] * 1e9 / repeat_inner )
puts "time: #{best[:duration] / repeat_inner} s, GFlops: #{perf}"

## Medium Kernel
### Definition
The medium kernel works using blocks of intermediate size. The medium kernel updates a block of $C$ of size $[kb,n]$ noted $\widetilde{C}$ using a block of $A$ of size $[mb,kb]$ noted $\widetilde{A}$ and a block of $B$ of of size $[kb,n]$ noted $\widetilde{B}$. $\widetilde{A}$ is stored as $mb/mr$ consecutive blocks of size $[mr, kb]$ in column major order while $\widetilde{C}$ is stored as $(mb/mr)*(n/nr)$ consecutive blocks of size $[mr,nr]$ in row major order and $\widetilde{B}$ is stored as $n/nr$ blocks of size $[kb, nr]$ in row major order.

In [ ]:
def medium_kernel(vector_length: 4, mb: nil, nr: nil, mr: nil, kb: nil)
  raise "nr must be a multiple of vector_length!" unless nr % vector_length == 0
  raise "mr must be a multiple of mb!" unless mb % mr == 0
  nvec = nr / vector_length
  nblocka = mb / mr

  inp = micro_kernel(vector_length: vector_length, nr: nr, mr: mr, kb: kb)
  
  n = Int :n, dir: :in
  nblockb = n/nr
  at = Real :ah, dim: [Dim(mr), Dim(kb), Dim(nblocka)], dir: :in, restrict: true
  bt = Real :bh, vector_length: vector_length, dim: [Dim(nvec), Dim(kb),  Dim(nblockb)], dir: :in, restrict: true
  ct = Real :ch, vector_length: vector_length, dim: [Dim(nvec), Dim(mr),  Dim(nblocka), Dim(nblockb)], dir: :inout, restrict: true
  medp = Procedure( "medp_#{vector_length}_#{mb}_#{nr}_#{mr}_#{kb}", [n, at, bt, ct] ) {
    jr = Int :jr
    ir = Int :ir
    decl jr, ir
    pr For(jr, 0, nblockb - 1) {
      pr For(ir, 0, nblocka - 1) {
        pr inp.procedure.call(at[0, 0, ir].address, bt[0, 0, jr].address, ct[0, 0, ir, jr].address)
      }
    }
  }
  k = CKernel::new(includes: "immintrin.h") {
    pr inp.procedure
    pr medp
  }
  k.procedure = medp
  k
end

### Test

In [ ]:
vector_length = 4
mr = 7
nr = 8
nblocka = 18
mb = mr * nblocka
kb = 480
nblockb = 1024
n  = nr * nblockb
nvec = nr / vector_length

type = type_map[get_default_real_size]
alignment = get_default_real_size*vector_length
a = NMatrix::new(type, kb, mb).random!
b = NMatrix::new(type, n, kb).random!
c = NMatrix::new(type, n, mb).random!

at = ANArray::new(type, alignment, mr, kb, nblocka)
bt = ANArray::new(type, alignment, vector_length, nvec, kb, nblockb)
ct = ANArray::new(type, alignment, vector_length, nvec, mr, nblocka, nblockb)
c_ref = ANArray::new(type, alignment, vector_length, nvec, mr, nblocka, nblockb)
at[true, true, true] = a.reshape(kb, mr, nblocka).transpose(1, 0, 2)[true, true, true]
bt[true, true, true, true] = b.reshape(vector_length, nvec, nblockb, kb)
                              .transpose(0, 1, 3, 2)[true, true, true, true]
ct[true, true, true, true, true] = c.reshape(vector_length, nvec, nblockb, mr, nblocka)
                              .transpose(0, 1, 3, 4, 2)[true, true, true, true, true]
c_ref[true, true, true, true, true] = (a*b + c).reshape(vector_length, nvec, nblockb, mr, nblocka)
                                         .transpose(0, 1, 3, 4, 2)[true, true, true, true, true]
nil

In [ ]:
p = medium_kernel(vector_length: vector_length, mb: mb, mr: mr, nr: nr, kb: kb)
p.run(n, at, bt, ct)

In [ ]:
max_error = (ct - c_ref).abs.max
raise "Computation error!" if max_error > 1e-8

In [ ]:
p.run(n, at, bt, ct)
repeat_inner = 10
res = 100.times.collect {
  p.run(n, at, bt, ct, repeat: repeat_inner)
}
best = res.min { |r1, r2|
  r1[:duration] <=> r2[:duration]
}
perf = mb * n * kb * 2 / (best[:duration] * 1e9 / repeat_inner )
puts "time: #{best[:duration] / repeat_inner} s, GFlops: #{perf}"